# Synthetic text PII Scanner

In [1]:
import torch
import os
import sys
os.environ["TOKENIZERS_PARALLELISM"] = "true"
sys.path.append("../..")

import examples.synthetic_data.aux.tab_data_aux as aux
from leakpro.synthetic_data_attacks.syn_text_pii_scanner import data_handling as dh
from leakpro.synthetic_data_attacks.syn_text_pii_scanner import utils
from leakpro.synthetic_data_attacks.syn_text_pii_scanner.pii_token_classif_models import ner_longformer_model as model_module

tab_path = '<PATH_TO_TAB_DATASET>'
syn_tab_path = '<PATH_TO_SYNTHETIC_TAB_DATASET>'
path_model = '<PATH_TO_MODEL_PII_CLASSIFIER>'


/opt/pyenv/versions/3.12.7/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Pipeline

Init tokenizer, label_set and data. Load data and model.

In [2]:
# Get tokenizer
tokenizer = model_module.get_tokenizer()
# Initialize label set and num_labels
label_set = dh.LabelSet(labels=['MASK'], IOB2_FORMAT=False)
num_labels = len(label_set.f_labels)
# Init data object with input and load data
data = utils.Data(
    ori = utils.SubData(
        # TAB data treatment function on original data
        path_or_data = aux.tab_data_treatment(
            tab_path = tab_path,
            only_first_annotator_flag = True,
            include_tasks_flag = False
        ),
        label_set = label_set,
        label_key = 'more_ex_label',
        batch_size = 8,
        shuffle = False,
        num_workers = 2
    ),
    syn = utils.SubData(
        path_or_data = syn_tab_path,
        batch_size = 8,
        shuffle = False,
        num_workers = 2
    )
)
utils.load_data(data=data, tokenizer=tokenizer)
# Set weights_crossentropy
weight_crossentr = 3
weights_crossentropy = torch.tensor([1.0] + [weight_crossentr*1.0 for _ in range(num_labels-1)], device=utils.device, dtype=torch.float32)

#Load model
model = model_module.load_model(
    num_labels = num_labels,
    weights_crossentropy = weights_crossentropy,
    crossentropy_ignore_index = -100,
    device = utils.device,
    path = path_model
)

Forward pass

In [3]:
#Forward pass
utils.forward_pass(
    data = data,
    num_labels = num_labels,
    model = model,
    verbose = True
)

Starting forward_pass with ori.


100%|██████████| 127/127 [02:44<00:00,  1.30s/it]


Ending forward_pass with ori.
Starting forward_pass with syn.


100%|██████████| 127/127 [03:00<00:00,  1.42s/it]


Ending forward_pass with syn.


Get PIIS and show first 20

In [4]:
inputs_ori = data.ori.model_input_output.inputs
labels_ori = data.ori.model_input_output.labels
predictions_ori = data.ori.model_input_output.predictions
inputs_syn = data.syn.model_input_output.inputs
predictions_syn = data.syn.model_input_output.predictions

#Get PIIs token offsets
ori_piis = utils.get_PIIs_01(
    labels = labels_ori,
    inputs = inputs_ori,
    tokenizer = tokenizer
)
ori_predicted_piis = utils.get_PIIs_01(
    labels = predictions_ori,
    inputs = inputs_ori,
    tokenizer = tokenizer
)
syn_piis = utils.get_PIIs_01(
    labels = predictions_syn,
    inputs = inputs_syn,
    tokenizer = tokenizer
)

#Show texts of ori_piis and ori_predicted_piis
print('\nComparison first 20 records ori_piis and ori_predicted_piis')
print('ori_piis\tori_predicted_piis')
for i in range(20):
    print(ori_piis[i].text, ori_predicted_piis[i].text, sep='\t')



Comparison first 20 records ori_piis and ori_predicted_piis
ori_piis	ori_predicted_piis
36244/06	36244/06
Mr Henrik Hasslund	Mr Henrik Hasslund
Mr Tyge Trier	Mr Tyge Trier
Ms Nina Holst-Christensen	Ms Nina Holst-Christensen
29366/03	29366/03
Mr D. Stępniak	Mr D. Stępniak
Mr J. Wołąsiewicz	Mr J. Wołąsiewicz
5138/04	5138/04
Mr Nusret Amutgan	Mr Nusret Amutgan
Ms B Özpolat	Ms B Özpolat
42596/98	42596/98
42603/98	42603/98
Mr Mustafa Sarı	Mr Mustafa Sarı
Ms Sibel Çolak	Ms Sibel Çolak
Mrs E. Çıtak	Mrs E. Çıtak
Mr Sarı	Mr Sarı
Ms Çolak	Ms Çolak
Ms Çolak	Ms Çolak
Mr Sarı	Mr Sarı
Mr Sarı	Mr Sarı


Detect non public PIIs and compare piis lists

In [5]:
similarity_threshold = 0.95
min_nr_repetitions = 1

#Get non public ori piis
non_public_ori_piis = utils.detect_non_public_pii(
    piis = ori_piis,
    similarity_threshold = similarity_threshold,
    min_nr_repetitions = min_nr_repetitions
)
#Get non public ori predicted piis
non_public_ori_predicted_piis = utils.detect_non_public_pii(
    piis = ori_predicted_piis,
    similarity_threshold = similarity_threshold,
    min_nr_repetitions = min_nr_repetitions
)
#Set ignore_list from predefined ignore list
ignore_list = aux.tab_predefined_ignore_piis_list + []

#Filter non_public_ori_piis from ignore_list
filtered_non_public_ori_piis = [i for i in non_public_ori_piis if i.text not in ignore_list]

sit, tot, sorted_sim_items, distr = utils.compare_piis_lists(
    ori_piis = filtered_non_public_ori_piis,
    syn_piis = syn_piis,
    similarity_threshold = similarity_threshold,
    verbose = True
)

print_ori_syn_cases_fun = aux.print_ori_syn_cases_fact(
    syn_piis = syn_piis,
    sorted_sim_items = sorted_sim_items,
    data = data
)


Start compare_piis_lists
Start print_distribution
Mean: 0.162158
0th Percentile: -0.328879
10th Percentile: 0.010996
100th Percentile: 1.000001
25th Percentile: 0.070178
50th Percentile: 0.144687
75th Percentile: 0.237819
90th Percentile: 0.339598
99th Percentile: 0.536478
End print_distribution
Nr. Similar Items: 135
Total Items: 87,481,260
Percentage: 0.000154%
End compare_piis_lists


In [6]:
print(f'\nOri piis len: {len(ori_piis)}')
print(f'Ori predicted piis len: {len(ori_predicted_piis)}')
print(f'Non public ori piis len: {len(non_public_ori_piis)}')
print(f'Non public ori predicted len: {len(non_public_ori_predicted_piis)}')
print(f'Filtered non public ori piis len: {len(filtered_non_public_ori_piis)}')
print(f'Nr synthetic PIIs: {len(syn_piis)}')
print(f'Len Ignore List: {len(ignore_list)}')

print(
    '\nlen sorted_sim_items: ',
    len(sorted_sim_items),
    ', sum len_syn_items: ',
    sum([v['len_syn_items'] for v in sorted_sim_items]),
    '\n',
    sep=''
)

for i in sorted_sim_items[0:20]:
    print(
        i['ori_item'], i['len_syn_items'], i['ori_doc_nr'],
        i['ori_text'], i['syn_docs'], i['len_syn_docs']
    )


Ori piis len: 15350
Ori predicted piis len: 15541
Non public ori piis len: 8945
Non public ori predicted len: 8990
Filtered non public ori piis len: 8564
Nr synthetic PIIs: 10215
Len Ignore List: 123

len sorted_sim_items: 110, sum len_syn_items: 135

7570 3 944 Mr. Koç [175, 934, 999] 3
7575 3 944 Mr. Koç [175, 934, 999] 3
7591 3 944 Mr. Koç [175, 934, 999] 3
7593 3 944 Mr. Koç [175, 934, 999] 3
1803 2 325 Mr P. Havers [339, 993] 2
2065 2 378 Mr Mehmet Aydın [493, 826] 2
5571 2 782 Mr N. Özdemir [340, 909] 2
5743 2 796 Mr Mehmet Güneş [301, 627] 2
21 2 5 Mr A. Erdoğan [869] 1
42 1 8 Süleyman Çetinkaya [935] 1
230 1 46 Mr M.S. Talay [968] 1
465 1 95 Mr Artur Nowak [25] 1
542 1 107 Mr S Karadayı [435] 1
560 1 112 Mr Dariusz Nowak [180] 1
593 1 120 Ms N. Vajić [103] 1
618 4 126 Singh [605] 1
657 1 135 87/2 [975] 1
663 1 137 Mr S. Sikora [405] 1
724 1 146 Mehmet Duman [376] 1
748 1 152 Mr C. Bîrsan [360] 1


In [7]:
print_ori_syn_cases_fun(0)


### PII Mr. Koç 

Original court case:
PROCEDURE

The case of Luedicke, Belkacem and Koç was referred to the Court by the Government of the Federal Republic of Germany (“the Government“) and the European Commission of Human Rights (“the Commission“). The case originated in three applications against the Federal Republic of Germany lodged with the Commission by Mr. Gerhard W. Luedicke, Mr. Mohammed Belkacem and Mr. Arif Koç in 1973, 1974 and 1975 respectively. The Commission ordered the joinder of these three applications on 4 October 1976.

Both the application of the Government, which referred to Article 48 (art. 48) of the Convention, and the request of the Commission, which relied on Articles 44 and 48, sub-paragraph (a) (art. 44, art. 48-a), and to which was attached the report provided for under Article 31 (art. 31), were lodged with the registry of the Court within the period of three months laid down in Articles 32 para. 1 and 47 ((art. 32-1, art. 47). The application was lodge